### Flask Api Profile Data

In [ ]:
from flask import Flask, request, jsonify
from sklearn.ensemble import RandomForestRegressor
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from flask_cors import CORS
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score

import nest_asyncio

nest_asyncio.apply()
app = Flask(__name__)
CORS(app) 

# Match for unique values
employmentStatus_mapping  = {
     "Unemployed" : 1,
     "Student" : 2,
     "Retired" : 2,
     "Employed part-time" : 3,
     "Employed full-time": 4,
     "Self-employed" : 5,
}

age_mapping= {
    "Over 50" : 1,
    "Under 18" : 2,
    "18-25" : 3,
    "26-35" : 4,
    "36-50" : 5,
    
}

investmentGoal_mapping = {
    "Wealth accumulation" : 1,
    "Retirement planning" : 2,
    "Education funding" : 3,
    "Buying a home" : 4,
    "Starting a business" : 5,
    "Other" : 0
}

pastInvestmentExperience_mapping = {
      "Yes" : 5,
      "No" : 1
}
 
financialMarketKnowledge_mapping = {
      "Very high" : 5,
      "High" : 4,
      "Moderate" : 3,
      "Low" : 2,
      "Very low" : 1
}

financialRiskComfortLevel_mapping   = {
      "Very high" : 5,
      "High" : 4,
      "Moderate" : 3,
      "Low" : 2,
      "Very low" : 1,
    
}

investmentProductKnowledge_mapping = { 
     "Very high" : 5,
     "High" : 4,
     "Moderate" : 3,
     "Low" : 2,
     "Very low" : 1
}

investmentSelectionCriteria_mapping = {
      "Expected return" : 5,
       "Diversification" :4 ,
       "Liquidity" : 3,
        "Others" : 2,
        "Risk tolerance" : 2,
        "Tax considerations" : 1,
       
}

avoidSpecificInvestments_mapping = {
      "Yes" : 5,
      "No" : 1
}

annualIncomeRange_mapping = {
      "Under 25,000 TND" : 1,
      "25,000 TND - 50,000 TND" : 2,
      "50,000 TND - 70,000 TND" : 3,
      "70,000 TND - 100,000 TND" : 4,
      "Over 100,000 TND" : 5
}

debts_mapping = {
      "Yes" : 5,
      "No" : 1
}

investmentTimeframe_mapping  = {
    "Very short term" : 1,
     "Short term" : 2,
     "Medium term" : 3,
     "Long term" : 4,
    "Very long term" : 5
}
investmentStyle_mapping = {
    "Vey conservative" : 1,
      "Conservative" : 2,
      "Moderate" : 3,
      "Aggressive" : 4,
     "Very aggressive" : 5
}
targetRateOfReturn_mapping  = {
      "0%-20%" : 1,
      "20%-40%" : 2,
      "40%-60%" : 3,
      "60%-80%" : 4,
      "80%-100%" : 5
}
preferredIndustries_mapping  = {
      "Technology" : 5,
      "Healthcare" : 4,
      "Finance" : 3,
      "Real Estate" : 2,
      "Consumer Goods" : 1,
      
     
}
liquidityPriority_mapping  = {
     "Very high priority" : 1,
      "High priority" : 2,
      "Medium priority" : 3,
      "Low priority" : 4,  #Des actifs moins liquides se réfèrent à des investissements qui ne peuvent pas être facilement vendus sur le marché ou convertis en espèces rapidement sans subir de perte significative de valeur ou rencontrer des difficultés à trouver des acheteurs.
       "Very low priority" : 5
}

reactionToMarketDownturn_mapping  = {
     "Stay invested and ride it out" : 5,
    "Increase investments" : 4,
     "Rebalance portfolio" : 3,
    "Sell investments and hold cash" : 2,
    "Others" : 1
}
comfortWithFluctuations_mapping = {
      "Very comfortable" : 5,
      "Comfortable" : 4,
      "Neutral" : 3,
      "Uncomfortable" : 2,
     "Very uncomfortable" : 1
}

sourceOfFunds_mapping  = {
      "Other" : 1,
      "Savings" : 2,
      "Salary" : 3,
      "Inheritance" : 4,
      "Investment returns" : 5
}


def fetch_profile_data(base_url, investor_id):
    api_url = f'{base_url}{investor_id}'
    response = requests.get(api_url)
    
    if response.status_code == 200:
        # Extract JSON data from the response
        data = response.json()
        
        # Create a Pandas DataFrame from JSON data
        ProfileData = pd.DataFrame([data])
        
        print("ProfileData for Investor id:", investor_id)
        print("The length of ProfileData is:", ProfileData.shape)
        return ProfileData
    else:
        print(f"Failed request {response.status_code}")
        return None


def CleaningData(ProfileData):
    ProfileData.fillna(0, inplace=True)
    return ProfileData


def apply_mapping_to_profile_data(profile_data):
    CleaningProfileData = CleaningData(profile_data)
    data = CleaningProfileData.replace({
        'employmentStatus': employmentStatus_mapping,
        'age': age_mapping,
        'investmentGoal': investmentGoal_mapping,
        'pastInvestmentExperience': pastInvestmentExperience_mapping,
        'financialMarketKnowledge': financialMarketKnowledge_mapping,
        'investmentProductKnowledge': investmentProductKnowledge_mapping,
        'investmentSelectionCriteria': investmentSelectionCriteria_mapping,
        'avoidSpecificInvestments': avoidSpecificInvestments_mapping,
        'annualIncomeRange': annualIncomeRange_mapping,
        'debts': debts_mapping,
        'investmentTimeframe': investmentTimeframe_mapping,
        'investmentStyle': investmentStyle_mapping,
        'targetRateOfReturn': targetRateOfReturn_mapping,
        'preferredIndustries': preferredIndustries_mapping,
        'liquidityPriority': liquidityPriority_mapping,
        'reactionToMarketDownturn': reactionToMarketDownturn_mapping,
        'comfortWithFluctuations': comfortWithFluctuations_mapping,
        'financialRiskComfortLevel': financialRiskComfortLevel_mapping,
        'sourceOfFunds': sourceOfFunds_mapping
    })

    return data

# Example usage:
# ProfileData_mapped = apply_mapping_to_profile_data(ProfileData)
# ProfileData_mapped

def apply_label_encoding(filename):
    # Load the dataset
    data = pd.read_csv(filename)
    
    # Apply label encoding using the mappings
    data['employment_status'] = data['employment_status'].map(employmentStatus_mapping)
    data['age'] = data['age'].map(age_mapping)
    data['investment_goal'] = data['investment_goal'].map(investmentGoal_mapping)
    data['past_investment_experience'] = data['past_investment_experience'].map(pastInvestmentExperience_mapping)
    data['financial_market_knowledge'] = data['financial_market_knowledge'].map(financialMarketKnowledge_mapping)
    data['investment_product_knowledge'] = data['investment_product_knowledge'].map(investmentProductKnowledge_mapping)
    data['investment_selection_criteria'] = data['investment_selection_criteria'].map(investmentSelectionCriteria_mapping)
    data['avoid_specific_investments'] = data['avoid_specific_investments'].map(avoidSpecificInvestments_mapping)
    data['annual_income_range'] = data['annual_income_range'].map(annualIncomeRange_mapping)
    data['debts'] = data['debts'].map(debts_mapping)
    data['investment_timeframe'] = data['investment_timeframe'].map(investmentTimeframe_mapping)
    data['investment_style'] = data['investment_style'].map(investmentStyle_mapping)
    data['target_rate_of_return'] = data['target_rate_of_return'].map(targetRateOfReturn_mapping)
    data['preferred_industries'] = data['preferred_industries'].map(preferredIndustries_mapping)
    data['liquidity_priority'] = data['liquidity_priority'].map(liquidityPriority_mapping)
    data['reaction_to_market_downturn'] = data['reaction_to_market_downturn'].map(reactionToMarketDownturn_mapping)
    data['comfort_with_fluctuations'] = data['comfort_with_fluctuations'].map(comfortWithFluctuations_mapping)
    data['financial_risk_comfort_level'] = data['financial_risk_comfort_level'].map(financialRiskComfortLevel_mapping)
    data['source_of_funds'] = data['source_of_funds'].map(sourceOfFunds_mapping)  
    
    return data

def CleaningData(data):
    data.fillna(0, inplace=True)
    return data

def preprocess_data_and_split(filename, target_column, test_size=0.2, random_state=42):
    
    # Load the dataset
    data_mapped = apply_label_encoding(filename)
    
    # Clean the data
    cleaned_data = CleaningData(data_mapped)
    
    # Separate features (X) and target variable (Y)
    X = cleaned_data.drop(columns=[target_column])
    Y = cleaned_data[target_column]

    # Encode categorical variables
    label_encoders = {}
    for column in X.columns:
        if X[column].dtype == 'object':
            label_encoders[column] = LabelEncoder()
            X[column] = label_encoders[column].fit_transform(X[column])

    # Split the dataset into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, Y_train, Y_test

 

def train_random_forest_model(X_train, Y_train, n_estimators=260, random_state=42):

    # Initialize and train the Random Forest model
    rf_regressor = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf_regressor.fit(X_train, Y_train)
    
    
    return rf_regressor


def predict_tolerance_risk(ProfileData_mapped, rf_regressor):

    investor_profile_data = ProfileData_mapped.drop(columns=['userId'])
    predicted_tolerance_risk = rf_regressor.predict(investor_profile_data)
    
    return predicted_tolerance_risk



# Define Flask endpoint
@app.route('/predict_tolerance_risk', methods=['POST'])
def predict_tolerance_risk_endpoint():
  
    request_data = request.json
    
    if request_data is None or 'base_url' not in request_data or 'investor_id' not in request_data:
        return jsonify({'error': 'Request data is missing or invalid.'}), 400
    
    
    ProfileData = fetch_profile_data(request_data['base_url'], request_data['investor_id'])
    activity = ProfileData['preferredIndustries'].values[0] if not ProfileData.empty else None
    print(activity)
    
    if ProfileData is None:
        return jsonify({'error': 'Failed to fetch profile data.'}), 500
    
    ProfileData_mapped = apply_mapping_to_profile_data(ProfileData)
 
    
    # Load trained model
    X_train, X_test, Y_train, Y_test = preprocess_data_and_split("C:\\Users\\Ranim\\Downloads\\ProfileDataset4.csv", "risk_tolerance")
    rf_regressor = train_random_forest_model(X_train, Y_train)
    
    if rf_regressor is None:
        return jsonify({'error': 'Failed to train the model.'}), 500
    
    # Predict tolerance risk
    predicted_risk = predict_tolerance_risk(ProfileData_mapped, rf_regressor)

    print('activity',activity)
    return jsonify({'predicted_tolerance_risk': predicted_risk.tolist() , 'activity': activity}), 200

# Run Flask app
if __name__ == '__main__':
    app.run(port=5004) 


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5004/ (Press CTRL+C to quit)
127.0.0.1 - - [08/May/2024 22:26:10] "OPTIONS /predict_tolerance_risk HTTP/1.1" 200 -


ProfileData for Investor id: 108
The length of ProfileData is: (1, 20)
Technology


127.0.0.1 - - [08/May/2024 22:26:12] "POST /predict_tolerance_risk HTTP/1.1" 200 -


activity Technology
ProfileData for Investor id: 108
The length of ProfileData is: (1, 20)
Technology


...

activity Technology
ProfileData for Investor id: 108
The length of ProfileData is: (1, 20)
Technology


127.0.0.1 - - [13/May/2024 13:48:03] "POST /predict_tolerance_risk HTTP/1.1" 200 -


activity Technology
ProfileData for Investor id: 108
The length of ProfileData is: (1, 20)
Technology


127.0.0.1 - - [13/May/2024 13:48:07] "POST /predict_tolerance_risk HTTP/1.1" 200 -


activity Technology
